In [1]:
# Dependencies
import json
import csv
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import datetime
from scipy.stats import linregress
import requests
from api_keys import omdb_api_key
from omdbapi.movie_search import GetMovie


In [3]:
# Create path
movie = GetMovie(api_key=omdb_api_key)


In [9]:
# Create Movies List

movie_list = ["The Shawshank Redemption", "The Godfather", 'Pulp Fiction',
'The Dark Knight',"Schindler's List",'Forrest Gump','The Matrix',
'Inception','The Lord of the Rings: The Return of the King','The Silence of the Lambs']


In [10]:
# Get movie details
for mov in movie_list:
    try:
        movinfo = movie.get_movie(title=mov)
        movinfo_df = pd.DataFrame(movinfo).append()

    except:
        print("movie not found")
    pass

movinfo_df

movie not found
movie not found
movie not found
movie not found
movie not found
movie not found
movie not found
movie not found
movie not found
movie not found


,title,year,rated,released,runtime,genre,director,writer,actors,plot,...,ratings,metascore,imdbrating,imdbvotes,imdbid,type,dvd,boxoffice,production,website
0,The Silence of the Lambs,1991,R,14 Feb 1991,118 min,"Crime, Drama, Thriller",Jonathan Demme,"Thomas Harris, Ted Tally","Jodie Foster, Anthony Hopkins, Lawrence A. Bonney",A young F.B.I. cadet must receive the help of ...,...,"{'Source': 'Internet Movie Database', 'Value':...",86,8.6,"1,490,803",tt0102926,movie,27 Aug 2015,"$130,742,922",N/A,N/A
1,The Silence of the Lambs,1991,R,14 Feb 1991,118 min,"Crime, Drama, Thriller",Jonathan Demme,"Thomas Harris, Ted Tally","Jodie Foster, Anthony Hopkins, Lawrence A. Bonney",A young F.B.I. cadet must receive the help of ...,...,"{'Source': 'Rotten Tomatoes', 'Value': '95%'}",86,8.6,"1,490,803",tt0102926,movie,27 Aug 2015,"$130,742,922",N/A,N/A
2,The Silence of the Lambs,1991,R,14 Feb 1991,118 min,"Crime, Drama, Thriller",Jonathan Demme,"Thomas Harris, Ted Tally","Jodie Foster, Anthony Hopkins, Lawrence A. Bonney",A young F.B.I. cadet must receive the help of ...,...,"{'Source': 'Metacritic', 'Value': '86/100'}",86,8.6,"1,490,803",tt0102926,movie,27 Aug 2015,"$130,742,922",N/A,N/A


In [4]:
# What year was 50 First Dates released?
movie = requests.get(url + "50 First Dates").json()
print(f'The movie 50 First Dates was released in {movie["Year"]}.')

The movie 50 First Dates was released in 2004.


In [5]:
# Who wrote Moana?
movie = requests.get(url + "Moana").json()
print(f'Moana was written by {movie["Writer"]}.')

Moana was written by Jared Bush (screenplay by), Ron Clements (story by), John Musker (story by), Chris Williams (story by), Don Hall (story by), Pamela Ribon (story by), Aaron Kandell (story by), Jordan Kandell (story by).


In [6]:
# What was the plot of the movie Sing?
movie = requests.get(url + "Sing").json()
print(f'The plot of Sing was: {movie["Plot"]}')

The plot of Sing was: In a city of humanoid animals, a hustling theater impresario's attempt to save his theater with a singing competition becomes grander than he anticipates even as its finalists' find that their lives will never be the same.
